In [4]:
import glob
import shutil
import os
import random
import numpy as np
import librosa as lr
import openl3 as l3
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [29]:
sc = ['dog_bark', 'gunshot', 'moving_motor_vehicle', 'sneeze_cough', 'footstep', 'keyboard', 'rain']
dev_path = 'data/dev/'
system_path = 'data/system/'

sci = 0

In [30]:
file_list = sorted(glob.glob(system_path+sc[sci]+'/*.wav'))

pruned_file_list = []
for idx, file_path in enumerate(file_list):
    audio, sr = lr.load(file_path)
    rms = lr.feature.rms(audio)
    # plt.plot(rms.T)
    # break
    crit = np.mean(rms)/np.median(rms)
    # print(crit)
    if crit>1.2:
        pruned_file_list.append(file_path)
    else:
        print('removing '+file_path)



print(len(pruned_file_list))

removing data/system/dog_bark/baseline002.wav
removing data/system/dog_bark/baseline006.wav
removing data/system/dog_bark/baseline007.wav
removing data/system/dog_bark/baseline013.wav
removing data/system/dog_bark/baseline025.wav
removing data/system/dog_bark/baseline032.wav
removing data/system/dog_bark/baseline035.wav
removing data/system/dog_bark/baseline036.wav
removing data/system/dog_bark/baseline037.wav
removing data/system/dog_bark/baseline040.wav
removing data/system/dog_bark/baseline047.wav
removing data/system/dog_bark/baseline049.wav
removing data/system/dog_bark/baseline050.wav
removing data/system/dog_bark/baseline052.wav
removing data/system/dog_bark/baseline053.wav
removing data/system/dog_bark/baseline056.wav
removing data/system/dog_bark/baseline065.wav
removing data/system/dog_bark/baseline074.wav
removing data/system/dog_bark/baseline075.wav
removing data/system/dog_bark/baseline076.wav
removing data/system/dog_bark/baseline078.wav
removing data/system/dog_bark/base

In [8]:
emb_size = 512

emb = np.zeros((len(file_list), emb_size))
for idx, file_path in tqdm(enumerate(pruned_file_list)):
    audio, sr = lr.load(file_path)
    e, ts = l3.get_audio_embedding(audio, sr, embedding_size=emb_size, verbose=0)
    emb[idx, :] = np.mean(e)


0it [00:00, ?it/s]

(100, 512)


perform kmeans with k = 7 for dev and 20 for system
select centroids

BQGF 3 noise types : lowpass with resampling, noise, distortion by clipping
GQBF choose 3 in random classes
GQGF randomly choose 3 in the 7 devs


In [6]:
import sklearn.cluster as cl    
from sklearn.neighbors import NearestNeighbors

nb_dev_sounds = 7
nb_system_sounds = 20

name = dev_path.split('/')

pck = np.load('data/emb/'+name[1]+'_'+sc[sci]+'.npy', allow_pickle=True)
data = pck.ravel()[0]['embeddings']
file_list = pck.ravel()[0]['file_list']

kmeans = cl.KMeans(n_clusters = nb_dev_sounds, random_state=0).fit(data)

In [14]:
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(data)
distances, indices = nbrs.kneighbors(kmeans.cluster_centers_)

u, c = np.unique(indices, return_counts=True)
if u[c > 1].size>0:
    print('duplicate medoids detected for ')

file = open('data/selection/'+name[1]+'_'+sc[sci]+'.txt','w')
for item in np.array(file_list)[indices]:
    file.write(item[0]+"\n")
file.close()

In [17]:
name = dev_path.split('/')

with open('data/selection/'+name[1]+'_'+sc[sci]+'.txt') as f:
    file_names = [line.rstrip('\n') for line in f]

print(file_names)

['data/dev/dog_bark/494.wav', 'data/dev/dog_bark/221.wav', 'data/dev/dog_bark/294.wav', 'data/dev/dog_bark/049.wav', 'data/dev/dog_bark/246.wav', 'data/dev/dog_bark/102.wav', 'data/dev/dog_bark/139.wav']


In [25]:
for k, f in enumerate(file_names):
    shutil.copyfile(f, 'data/audio/'+sc[sci]+'/reference_'+str(k)+'.wav')

GQGF randomly choose 3 in the 7 devs

In [73]:
for dir in sc:
    os.makedirs('data/stimuli/'+dir)

In [24]:
for k in range(3):
    shutil.copyfile(file_names[k], 'data/audio/'+sc[sci]+'/gqgf'+str(k)+'.wav')

GQBF choose 3 in random classes

In [44]:
import copy

scr = copy.deepcopy(sc)
for k in range(3):
    scnis = list(range(len(scr)))
    scnis.remove(sci)
    scni = random.choice(scnis)

    with open('data/selection/dev_'+scr[scni]+'.txt') as f:
        file_names = [line.rstrip('\n') for line in f]

    shutil.copyfile(random.choice(file_names), 'data/audio/'+scr[sci]+'/gqbf'+str(k)+'.wav')
    scr.remove(scr[scni])

['dog_bark', 'gunshot', 'moving_motor_vehicle', 'sneeze_cough', 'footstep', 'keyboard', 'rain']
1
['dog_bark', 'moving_motor_vehicle', 'sneeze_cough', 'footstep', 'keyboard', 'rain']
5
['dog_bark', 'moving_motor_vehicle', 'sneeze_cough', 'footstep', 'keyboard']
2


BQGF 3 noise types : lowpass with resampling, noise, distortion by clipping

In [70]:
import soundfile as sf
import IPython

for k in range(3):
    shutil.copyfile(file_names[3+k], 'data/audio/'+sc[sci]+'/gqgf'+str(k)+'.wav')

audio, sr = lr.load(file_names[3])
factor = 6
audio = lr.resample(audio, sr, sr//factor)
audio = lr.resample(audio, sr, sr*factor)
sf.write('data/audio/'+sc[sci]+'/gqgf_0.wav', audio, sr)


audio, sr = lr.load(file_names[4])
factor = 20
audio = audio+np.random.randn(audio.shape[0])*np.max(np.abs(audio))/factor
sf.write('data/audio/'+sc[sci]+'/gqgf_1.wav', audio, sr)

audio, sr = lr.load(file_names[5])
factor = 20
audio = np.clip(audio, np.min(audio)/factor, np.max(audio)/factor)
sf.write('data/audio/'+sc[sci]+'/gqgf_2.wav', audio, sr)

IPython.display.Audio(audio, rate=sr, autoplay=True)